In [157]:
import re
import requests
import json
import sqlite3
import glob
import urllib.parse
from bs4 import BeautifulSoup

In [ ]:
import json
import sqlite3

conn = None

#  データベースに接続する
def connect():
    global conn
    conn = sqlite3.connect('./sample.db')

def close():
    conn.close()
    
def create_table():
    conn.execute('DROP TABLE IF EXISTS countries')
    conn.execute("""CREATE TABLE countries (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            country_name text,
            content text
            )""")
# def create_table():
#     conn.execute('DROP TABLE IF EXISTS docs')
#     conn.execute('''CREATE TABLE IF EXISTS docs (
#             id        INTEGER PRIMARY KEY AUTOINCREMENT,
#             content   TEXT,
#             meta_info BLOB,
#             sentence  BLOB,
#             chunk     BLOB,
#             token     BLOB
#             )''')

#  Countriesテーブルにレコードを挿入
def load(name, content):
    conn.execute(
        'INSERT INTO countries (country_name, content) VALUES (?, ?)', (name, content))
    conn.commit()
    
def get(doc_id, fl):
    row_ls = conn.execute(
        'SELECT {} FROM docs WHERE id = ?'.format(','.join(fl)), (doc_id,)).fetchone()
    row_dict = {}
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict

def get_all_contents(limit, offset=0):
    return [record[0] for record in 
               conn.execute(
            'SELECT content FROM countries LIMIT ? OFFSET ?',
            (limit, offset))]


In [ ]:
if __name__ == "__main__":
    connect()
    create_table()
    close()

In [ ]:
url = "https://ja.wikipedia.org/wiki/%E5%9B%BD%E3%81%AE%E4%B8%80%E8%A6%A7"
header = "https://ja.wikipedia.org"

r = requests.get(url)
bs = BeautifulSoup(r.text, "html.parser")
table = bs.find("table", class_="wikitable")

connect()
create_table()
for tr in table.find_all("tr"):
    a = tr.find_all("a")
    if a:
        name = a[1].text
        link = a[1].get("href")
        res = requests.get(header + link)
        load(name, res.text)
close()

In [155]:
connect()
html = get_all_contents(limit=1)
soup = BeautifulSoup(html[0], "html.parser")
close()

In [173]:
content = soup.find(id="content").text
re.sub("[[^\x01-\x7E]|\w]+", "", content)

'アイスランド出典フリー百科事典『ウィキペディア（）』ナビゲーションに移動検索に移動アイスランド共和国Í（国旗）国章国の標語：なし国歌：ö（アイスランド語）賛美歌公用語アイスランド語首都レイキャビク最大の都市レイキャビク政府大統領グズニ・ヨハンネソン首相カトリーン・ヤコブスドッティル面積総計（位）水面積率人口総計（年）人（位）人口密度人（自国通貨表示）合計（年）億アイスランド・クローナ合計（年）ドル（位）合計（年）ドル（位）人あたりドル独立デンマークから年月日通貨アイスランド・クローナ時間帯±（：なし）国際電話番号アイスランドは、北ヨーロッパの北大西洋上に位置する共和制を取る国家である。首都はレイキャビク。総人口は約人。目次概要国名歴史植民と自然への影響新天地と自治の終焉独立運動政治加盟問題背景加盟申請まで加盟交渉開始加盟申請取り下げエネルギー政策捕鯨軍事地方行政区分の市地理火山主な湖主な山気候都市と交通経済国家経済金融危機以前金融危機以後資源漁業エネルギー製造業商業貿易国民人口統計言語宗教教育人名同性婚文化食文化文学スポーツ音楽世界遺産祝祭日出身人物映画監督音楽家スポーツ選手起業家脚注参考文献関連項目外部リンク概要編島国であり、グリーンランドの南東方、ブリテン諸島やデンマークの自治領であるフェロー諸島の北西に位置する。アイスランド島が主な領土であり、イギリスとのタラ戦争の舞台にもなった漁業基地であるヴェストマン諸島、北極圏上にあるグリムセイ島などの周辺の島嶼（とうしょ）も領有する。高緯度にあるためメルカトル図法の地図では広大な島のように描かれるが、実際の面積はと、フィリピンのルソン島（）や、大韓民国（）とほぼ同じである。グレートブリテン島（約万）の約半分、または日本の北海道と四国を合わせた程度の面積である。多くの火山が存在し、温泉も存在する他、豊富な地熱を発電などに利用している。一方で、噴火による災害も多い（アイスランドの火山活動も参照）。例えば、年にはエイヤフィヤトラヨークトル氷河の火山が噴火し、欧州を中心に世界中で航空機の運用に大きな影響を与えた。なお、国土の大部分が西経度よりも西にあるにもかかわらず、グリニッジ標準時を使っている。沿岸には多数のフィヨルドがある。火山性の土壌で大地は肥沃とは言えない。また、かつてアイスランドの森林は乱伐が行われたことも手伝って